In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from financial.momentum.models.kerasAdvanced import KerasAdvancedModelFactory
from financial.momentum.experiment.modelExperiment import ModelExperimentFactory
import os
from dotenv import load_dotenv
from financial.momentum.utilities import find_dotenv
from financial.momentum.utilities import metrics
import financial.data as fd
from financial.io.file.cache import FileCache
import pandas as pd
from financial.lab.experiment import Experiment
from sklearn.metrics import r2_score
from financial.model import Model

import tensorflow as tf
from keras import backend as K
import gc

def reset_gpu():
    K.clear_session()
    tf.keras.backend.clear_session()
    gc.collect()

2025-07-20 13:10:09.559688: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-20 13:10:09.569202: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753009809.580386    7744 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753009809.583840    7744 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753009809.592496    7744 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [3]:
load_dotenv(dotenv_path=find_dotenv())

cache = os.environ["CACHE"]
model_path = os.environ["MODEL"]
print(model_path)
print(os.environ["DATA"])
datastore = fd.CachedDataStore(path=os.environ["DATA"], cache=FileCache(cache_path=cache+"/"))
print(datastore) 

/home/manidmt/TFG/OTRI/models/keras
/home/manidmt/TFG/OTRI/data
CachedDataStore with 960 data sources [cache stats: {'size': 0, 'hit': 0, 'miss': 0, 'write': 0, 'read': 0, 'update': 0}]


In [11]:
ticker = "MSFT"
start_date = "2020-01-01"
end_date = "2025-06-30"

In [5]:
data = datastore.get_data(ticker, start_date, end_date)
print(data)
target = data[90 + 20:]
print(target.size)

Date
1990-01-02      0.377807
1990-01-03      0.379936
1990-01-04      0.391110
1990-01-05      0.381532
1990-01-08      0.387385
                 ...    
2025-06-24    490.109985
2025-06-25    492.269989
2025-06-26    497.450012
2025-06-27    495.940002
2025-06-30    497.410004
Name: MSFT, Length: 8939, dtype: float64
8829


In [6]:
factory = KerasAdvancedModelFactory()

In [16]:
from itertools import product

# Arquitecturas a probar
architectures = ["cnn"]
#architectures = ["rnn", "lstm", "cnn", "cnn2d", "transformer"]

# Hiperparámetros a explorar
layer_option = [64, 32]
activation = ["relu"]
batch_size = 32
epochs = 10

def ticker_single_models(ticker):

    data = datastore.get_data(ticker, start_date, end_date)    
    target = data[90 + 20:]
    
    for arch in architectures:
        name = f"{arch}_{ticker}_{end_date[:4]}_multiple"
        path = model_path + "/" + name

        try:
            if Model.from_file(name, path=os.environ["MODEL"]):
                print(f"⚠️ Modelo {name} ya existe. Saltando...")
                continue
        except FileNotFoundError:
            print(f"📂 Modelo {name} no existe. Procediendo a entrenar...")

        config = {
            "mode": "global",  
            "datastore": datastore,
            "ticker": [ticker, "^GSPC"],
            "model_factory": factory,
            "name": name,
            "start_year": start_date,
            "end_year": end_date,
            "lookahead": 20,
            "horizon": 90,
            "model_params": {
                "architecture": arch,
                "topology": {
                    "layers": layer_option,
                    "activation": {
                        "hidden": activation,
                        "output": "linear"
                    }
                },
                "optimization": {
                    "optimizer": "adam",
                    "loss": "mean_squared_error",
                    "metrics": ["mae"],
                    "epochs": epochs,
                    "batch_size": batch_size,
                    "validation_split": 0.1
                }
            }
        }

        try:
            print("RUNNING MODEL:")
            experiment = ModelExperimentFactory.create_experiment(config)
            experiment.run()
            print("METRICS:")
            preds = experiment.reconstruct_absolute_predictions_from_relative()
            keras_model = experiment.final_model.model  # esto es keras.Model
            keras_model.save(path + "_f.keras")
            # Guardar predicciones
            preds = experiment.reconstruct_absolute_predictions_from_relative()
            preds.to_csv(path + "_preds.csv")
            print(f"✅ Predicciones guardadas en: {path}")
            metrics(experiment, preds, target, os.environ["MODEL"])
            reset_gpu()
        except Exception as e:
            print(f"Falló el modelo {name}: {e}")

In [17]:
ticker_single_models("GOOG")
ticker_single_models("BLK")
ticker_single_models("BAM")

📂 Modelo cnn_GOOG_2025_multiple no existe. Procediendo a entrenar...
RUNNING MODEL:
Architecture: cnn
Epoch 1/300
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.0025 - loss: 0.9365  
Epoch 2/300
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0025 - loss: 0.4071
Epoch 3/300
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0012 - loss: 0.2119     
Epoch 4/300
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 5.1128e-04 - loss: 0.0619
Epoch 5/300
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 9.0191e-04 - loss: 0.0302
Epoch 6/300
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0016 - loss: 0.0283     
Epoch 7/300
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0016 - loss: 0.0142     
Epoch 8/300
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 6.8490e-04 - loss: 0.0085 
Epoch 9/300
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 9.0191e-04 - loss: 0.0076 
Epoch 10/300
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 5.1128e-04 - loss: 0.0058
Epoch 11/300
8/8 ━━━━━━━━

29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Epoch 1/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.0020 - loss: 0.6073  
Epoch 2/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 5.4953e-04 - loss: 0.1065 
Epoch 3/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 4.4103e-04 - loss: 0.0396 
Epoch 4/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 5.4953e-04 - loss: 0.0168 
Epoch 5/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0011 - loss: 0.0091
Epoch 6/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 2.6664e-04 - loss: 0.0069
Epoch 7/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 5.4953e-04 - loss: 0.0056
Epoch 8/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0020 - loss: 0.0054
Epoch 9/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0014 - loss: 0.0058
Epoch 10/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.0014 - loss: 0.0057
Epoch 11/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 1

41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
METRICS:
✅ Predicciones guardadas en: /home/manidmt/TFG/OTRI/models/keras/cnn_GOOG_2025_multiple
GLOBAL:
n=1270 MSE=101.7335 RMSE=10.0863 MAE=7.6181 MAPE=0.0599 R² = 0.9012
TRAIN: 
n=916 MSE=0.0082 RMSE=0.0905 MAE=0.0763 MAPE=3.5239
TEST: 
n=374 MSE=0.0117 RMSE=0.1083 MAE=0.0970 MAPE=4.1249

Métricas guardadas exitosamente en: /home/manidmt/TFG/OTRI/models/keras/cnn_GOOG_2025_multiple_metrics.json
📂 Modelo cnn_BLK_2025_multiple no existe. Procediendo a entrenar...
RUNNING MODEL:
Architecture: cnn
Epoch 1/300
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0000e+00 - loss: 0.5571  
Epoch 2/300
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 0.1291 
Epoch 3/300
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0000e+00 - loss: 0.0532 
Epoch 4/300
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0000e+00 - loss: 0.0417 
Epoch 5/300
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.000

29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Epoch 1/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 0.4931  
Epoch 2/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 0.0708 
Epoch 3/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 0.0288 
Epoch 4/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 0.0142 
Epoch 5/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0000e+00 - loss: 0.0085
Epoch 6/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 0.0064 
Epoch 7/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 0.0062 
Epoch 8/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 0.0061 
Epoch 9/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 0.0062 
Epoch 10/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 0.0070 
Epoch 11/300
11/11 ━

41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
METRICS:
✅ Predicciones guardadas en: /home/manidmt/TFG/OTRI/models/keras/cnn_BLK_2025_multiple
GLOBAL:
n=1270 MSE=2739.8364 RMSE=52.3434 MAE=41.3330 MAPE=0.0590 R² = 0.8603
TRAIN: 
n=916 MSE=0.0085 RMSE=0.0924 MAE=0.0705 MAPE=6.1512
TEST: 
n=374 MSE=0.0039 RMSE=0.0623 MAE=0.0455 MAPE=4.0239

Métricas guardadas exitosamente en: /home/manidmt/TFG/OTRI/models/keras/cnn_BLK_2025_multiple_metrics.json
📂 Modelo cnn_BAM_2025_multiple no existe. Procediendo a entrenar...
RUNNING MODEL:
Architecture: cnn
Epoch 1/300
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0000e+00 - loss: 0.6895  
Epoch 2/300
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0000e+00 - loss: 0.3317 
Epoch 3/300
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0000e+00 - loss: 0.1333 
Epoch 4/300
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0000e+00 - loss: 0.0441 
Epoch 5/300
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.00

29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Epoch 1/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 0.7579  
Epoch 2/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 0.1757 
Epoch 3/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0000e+00 - loss: 0.0774
Epoch 4/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 0.0373 
Epoch 5/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 0.0177 
Epoch 6/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0000e+00 - loss: 0.0155
Epoch 7/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 0.0136 
Epoch 8/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 0.0136 
Epoch 9/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 0.0125 
Epoch 10/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 0.0119 
Epoch 11/300
11/11 ━━

41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
METRICS:
✅ Predicciones guardadas en: /home/manidmt/TFG/OTRI/models/keras/cnn_BAM_2025_multiple
GLOBAL:
n=1270 MSE=16.8880 RMSE=4.1095 MAE=3.0213 MAPE=0.0732 R² = 0.8258
TRAIN: 
n=916 MSE=0.0135 RMSE=0.1164 MAE=0.0772 MAPE=2.7974
TEST: 
n=374 MSE=0.0079 RMSE=0.0888 MAE=0.0712 MAPE=1.9049

Métricas guardadas exitosamente en: /home/manidmt/TFG/OTRI/models/keras/cnn_BAM_2025_multiple_metrics.json


In [ ]:
index_tickers = [
    "^GSPC",          # S&P 500
    "^IXIC",          # Nasdaq
    "^DJI",           # Dow Jones Industrial Average
    "^N225",          # Nikkei 225
    "^IBEX",          # Ibex 35
    "AMZN",
    "TSLA",
]

In [ ]:
for ticker in index_tickers:
    ticker_single_models(ticker)